In [1]:
# https://gogul.dev/software/hand-gesture-recognition-p1
# Soustraction d'arrière-plan
# Détection de mouvement et seuillage
# Extraction de contour

In [1]:
# organize imports
import cv2 as cv
import imutils
import numpy as np

# global variables
bg = None

In [2]:
#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv.accumulateWeighted(image, bg, aWeight)

In [3]:
#---------------------------------------------
# To segment the region of hand in the image
#---------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv.threshold(diff, threshold, 255, cv.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv.findContours(thresholded.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv.contourArea)
        return (thresholded, segmented)

In [5]:
#-----------------
# MAIN FUNCTION
#-----------------
# initialize weight for running average
aWeight = 0.5

# get the reference to the webcam
camera = cv.VideoCapture(0)

# region of interest (ROI) coordinates
top, right, bottom, left = 10, 350, 225, 590

# initialize num of frames
num_frames = 0

# keep looping, until interrupted
while(True):
    # get the current frame
    (grabbed, frame) = camera.read()

    # resize the frame
    frame = imutils.resize(frame, width=700)

    # flip the frame so that it is not the mirror view
    frame = cv.flip(frame, 1)

    # clone the frame
    clone = frame.copy()

    # get the height and width of the frame
    (height, width) = frame.shape[:2]

    # get the ROI
    roi = frame[top:bottom, right:left]

    # convert the roi to grayscale and blur it
    gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (7, 7), 0)

    # to get the background, keep looking till a threshold is reached
    # so that our running average model gets calibrated
    if num_frames < 30:
        run_avg(gray, aWeight)
    else:
        # segment the hand region
        hand = segment(gray)

        # check whether hand region is segmented
        if hand is not None:
            # if yes, unpack the thresholded image and
            # segmented region
            (thresholded, segmented) = hand

            # draw the segmented region and display the frame
            cv.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
            cv.imshow("Thesholded", thresholded)

    # draw the segmented hand
    cv.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

    # increment the number of frames
    num_frames += 1

    # display the frame with segmented hand
    cv.imshow("Video Feed", clone)

    # observe the keypress by the user
    keypress = cv.waitKey(1) & 0xFF

    # if the user pressed "q", then stop looping
    if keypress == ord("q"):
        break

# free up memory
camera.release()
cv.destroyAllWindows()